# Mini Competition: Richter's Predictor

Import modules and data

In [364]:
# Modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [365]:
# Import scripts
import helper_functions # various helper functions
import log_regression # Simple regression model
import lgb_optimized # Random forest

In [366]:
# Load data
X, y, X_test = helper_functions.imports()

### Data Analysis

Start with a basic analysis of the data to get a general feeling of what we are dealing with.

In [367]:
print(f"Proportions of the DataFrame X, containing the features for testing: {X.shape}")
print(f"Proportions of the DataFrage y, containing the target value for testing: {y.shape}")
print(f"Proportions of the DataFrame X, containing the features for the prediction: {X_test.shape}")

Proportions of the DataFrame X, containing the features for testing: (260601, 39)
Proportions of the DataFrage y, containing the target value for testing: (260601, 2)
Proportions of the DataFrame X, containing the features for the prediction: (86868, 39)


In [368]:
X.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [369]:
y.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [370]:
X_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


Check whether X and X_test have the same columns

In [371]:
X.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_i

In [372]:
helper_functions.test_column_equality(X, X_test)

Both DataFrames have the same columns.


In [373]:
X.dtypes

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

In [374]:
X.nunique()

building_id                               260601
geo_level_1_id                                31
geo_level_2_id                              1414
geo_level_3_id                             11595
count_floors_pre_eq                            9
age                                           42
area_percentage                               84
height_percentage                             27
land_surface_condition                         3
foundation_type                                5
roof_type                                      3
ground_floor_type                              5
other_floor_type                               4
position                                       4
plan_configuration                            10
has_superstructure_adobe_mud                   2
has_superstructure_mud_mortar_stone            2
has_superstructure_stone_flag                  2
has_superstructure_cement_mortar_stone         2
has_superstructure_mud_mortar_brick            2
has_superstructure_c

From the description, we assume Geo Level 3 is the most precise whereas Geo Level 1 the least precise.
Follow that logic, we expect more unique data points in Level 3 than Level 1.

In [375]:
print(f"Unique data points in geo_level_1_id: {X.loc[:, 'geo_level_1_id'].nunique()}")
print(f"Unique data points in geo_level_2_id: {X.loc[:, 'geo_level_2_id'].nunique()}")
print(f"Unique data points in geo_level_3_id: {X.loc[:, 'geo_level_3_id'].nunique()}")

Unique data points in geo_level_1_id: 31
Unique data points in geo_level_2_id: 1414
Unique data points in geo_level_3_id: 11595


## Data Cleaning

In [376]:
# list of columns that shall be dropped at the end
drop_cols = []

# helper functions
def ids():
    ids = [id for id in globals().keys() if not id.startswith("_")]
    print(ids[5:])
ids()

print(X.shape, 'damage_grade' in X.columns)
data = pd.merge(left=y, right=X, on='building_id', how='inner')
print(data.shape, 'damage_grade' in data.columns) #ok, one column was added

cols = data.columns
for col in cols:
    globals()[col] = col



['open', 'pd', 'train_test_split', 'f1_score', 'helper_functions', 'log_regression', 'lgb_optimized', 'X', 'y', 'X_test', 'drop_cols', 'ids', 'data', 'cols', 'col', 'building_id', 'damage_grade', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'legal_ownership_status', 'count_families', 'has_secondary_use', 'has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_

### `geo_level2` mean encodings

In [377]:
geo_levels = []
for col in filter(lambda col: col.startswith("geo"), X):
    print(str(col))
    geo_levels.append(col)

geo_level_1_id
geo_level_2_id
geo_level_3_id


In [378]:
mean_encodings = data.groupby('geo_level_2_id')['damage_grade'].mean()
pd.DataFrame(mean_encodings).head()

,damage_grade
geo_level_2_id,
0,2.763158
1,2.348039
3,2.103896
4,2.520635
5,2.160000


In [379]:
for df in X, X_test:
    df["geo_level_2_enc"] = df['geo_level_2_id'].map(mean_encodings)
    # print(df[["geo_level_2_id", "geo_level_2_enc"]].sort_values("geo_level_2_id")[:100], end="\n")

print(geo_levels)

['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']


### `geo_level1` dummies

In [380]:
print(dummies.columns, dummies.shape, sep="\n")


Index(['geo_level_cat_0', 'geo_level_cat_1', 'geo_level_cat_2',
       'geo_level_cat_3', 'geo_level_cat_4', 'geo_level_cat_5',
       'geo_level_cat_6', 'geo_level_cat_7', 'geo_level_cat_8',
       'geo_level_cat_9', 'geo_level_cat_10', 'geo_level_cat_11',
       'geo_level_cat_12', 'geo_level_cat_13', 'geo_level_cat_14',
       'geo_level_cat_15', 'geo_level_cat_16', 'geo_level_cat_17',
       'geo_level_cat_18', 'geo_level_cat_19', 'geo_level_cat_20',
       'geo_level_cat_21', 'geo_level_cat_22', 'geo_level_cat_23',
       'geo_level_cat_24', 'geo_level_cat_25', 'geo_level_cat_26',
       'geo_level_cat_27', 'geo_level_cat_28', 'geo_level_cat_29',
       'geo_level_cat_30'],
      dtype='object')
(86868, 31)


In [ ]:
for df in X, X_test:
    dummies = pd.get_dummies(df["geo_level_1_id"], prefix="geo_level_cat")
    df = pd.concat([df, dummies], axis=1)
    print(df.columns, df.shape, end="\n")

In [344]:
geo_columns = [col for col in X.columns if col.startswith('geo') and col!="geo_level_1_id"]
# print(geo_columns)
X.groupby("geo_level_1_id").first()[geo_columns]
drop_cols.extend(geo_levels)
X.columns


### drop categorical variables for now, I will encode them next

In [345]:
object_dtype_cols = [
    "land_surface_condition", "foundation_type", "roof_type", "ground_floor_type", 
    "other_floor_type", "position", "plan_configuration", "legal_ownership_status"
]
# 

drop_cols.extend(object_dtype_cols)
drop_cols

['geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [297]:
# Outliers 

### Prepare data for analysis

In [298]:
# set index to building_id
for df in y, X, X_test:
    df.set_index("building_id", inplace=True)


In [299]:
# drop columns
print("drop_cols:", drop_cols)
for df in X, X_test:
    df.drop(columns=drop_cols, inplace=True)
    print(df.shape, end="\n")

drop_cols: ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']
(260601, 28)
(86868, 28)


In [305]:

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_pred = X_test

# for k,v in {"X":X, X_train, X_valid, X_test}:
for name,df in {"X":X, "X_train":X_train, "X_valid": X_valid, "X_test": X_test}.items():
    print(name, *df.shape, sep="\t")

geo_columns = [col for col in X.columns if col.startswith("geo")]
print(X[geo_columns].columns)



X	260601	28
X_train	208480	28
X_valid	52121	28
X_test	86868	28
Index(['geo_level_2_enc'], dtype='object')


Visualization

## Models

Simple estimation using regressionm

In [301]:
lr = log_regression.lr(X_train, y_train)
y_valid_pred_lr = lr.make_prediction(X_valid)

Generation of Random Forest and optimization

In [115]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [116]:
import lgb_optimized
# Initiate and optimize model
lgb = lgb_optimized.LGBM(X_train, X_valid, y_train, y_valid)

[I 2023-10-07 09:56:57,659] A new study created in memory with name: no-name-1e8efe33-e5a6-4845-accc-7d83fff3a8b3
[I 2023-10-07 09:57:02,017] Trial 0 finished with value: 0.15820878340783945 and parameters: {'learning_rate': 0.07731410399795237, 'subsample': 0.8043970710753758, 'num_leaves': 81, 'min_data_in_leaf': 6, 'max_depth': 3, 'lambda_l2': 0.16384976626764136}. Best is trial 0 with value: 0.15820878340783945.
[I 2023-10-07 09:57:08,509] Trial 1 finished with value: 0.15201166516375358 and parameters: {'learning_rate': 0.2515694756623492, 'subsample': 0.8758879728223203, 'num_leaves': 81, 'min_data_in_leaf': 18, 'max_depth': 8, 'lambda_l2': 0.724990270046077}. Best is trial 0 with value: 0.15820878340783945.


Number of finished trials: 2
Best trial:
--------------------------------
Best F1 Score: 0.15820878340783945
--------------------------------
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] lambda_l2 is set=0.16384976626764136, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.16384976626764136
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] lambda_l2 is set=0.16384976626764136, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.16384976626764136
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062453 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 512
[LightGBM] [Info] Number of data points in the train set: 260601, number of used featur

In [117]:
y_pred_lgb = lgb.make_prediction(X_valid)

[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] lambda_l2 is set=0.16384976626764136, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.16384976626764136


#### Evaluation

f1 Score using Logistic Regression

In [118]:
f1_lr = f1_score(y_valid, y_valid_pred_lr, average='macro')
f1_lr

0.5731212972136793

f1 Score for Random Forest

In [119]:
f1_lgb = f1_score(y_valid, y_pred_lgb, average='macro')
f1_lgb

0.643212105529923

### Create prediction on testset

In [120]:
y_pred_lgb = lgb.make_prediction(X_test_pred)

ValueError: Number of features of the model must match the input. Model n_features_ is 59 and input n_features is 28

Export

In [ ]:
# Export prediction to create csv by DrivenData.org datastandards
X_test.reset_index(inplace=True)

helper_functions.write_output(X_test, y_pred_lgb)

In [ ]:
X.columns

Index(['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'geo_level_2_enc'],
      dtype='object')

In [24]:
import session_info
session_info.show()